In [ ]:
import pandas as pd
import numpy as np
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

In [ ]:
# Load the dataset
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)

In [ ]:
# Preprocess the text
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Convert to lowercase
    text = text.lower()
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

In [ ]:
# Apply the preprocessing function to the 'headline' column
data['headline'] = data['headline'].apply(preprocess_text)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['headline'], data['is_sarcastic'], test_size=0.3, random_state=42)


In [ ]:
# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)


In [ ]:
# Pad sequences
max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

In [ ]:
# Build the model
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
468/468 [==============================] - 361s 750ms/step - loss: 0.4270 - accuracy: 0.7923 - val_loss: 0.3409 - val_accuracy: 0.8513
Epoch 2/10
468/468 [==============================] - 350s 748ms/step - loss: 0.1543 - accuracy: 0.9444 - val_loss: 0.3877 - val_accuracy: 0.8468
Epoch 3/10
468/468 [==============================] - 353s 753ms/step - loss: 0.0572 - accuracy: 0.9808 - val_loss: 0.6318 - val_accuracy: 0.8273
Epoch 4/10
468/468 [==============================] - 329s 704ms/step - loss: 0.0226 - accuracy: 0.9921 - val_loss: 0.7711 - val_accuracy: 0.8297
Epoch 5/10
468/468 [==============================] - 331s 705ms/step - loss: 0.0085 - accuracy: 0.9976 - val_loss: 1.2529 - val_accuracy: 0.8291
Epoch 6/10
468/468 [==============================] - 327s 699ms/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 0.9805 - val_accuracy: 0.8217
Epoch 7/10
468/468 [==============================] - 327s 698ms/step - loss: 0.0094 - accuracy: 0.9971 - val_loss: 1.2356 -

In [ ]:
# Evaluate the model
y_pred_prob = model.predict(X_test_pad)
y_pred = np.round(y_pred_prob).astype(int)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Accuracy:', accuracy)
print('F1-score:', f1)

251/251 [==============================] - 46s 175ms/step
Accuracy: 0.8207912142768002
F1-score: 0.7898741586186714
